<a href="https://colab.research.google.com/github/TamizharasanG/Travel-and-Tourism-Clarity-TTS/blob/main/Clarity_TTS_Data_Preperation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# NLP packages
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


***Importing dataset from Kaggle*** - *Indian Tourism data*

---

Uploadd the jaon file to colab before running the code. The json file is attached in my repo.

In [7]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download ritvik1909/indian-places-to-visit-reviews-data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
indian-places-to-visit-reviews-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
data = pd.read_csv("/content/indian-places-to-visit-reviews-data.zip")

In [8]:
data.shape

(1482466, 7)

In [9]:
data.head()

,City,Place,Review,Rating,Name,Date,Raw_Review
0,Aamby Valley City,19 Degree North,aamby valley beautiful place clear blue skies ...,5,Anonymous,NaN,Aamby valley is a beautiful place with its cle...
1,Aamby Valley City,19 Degree North,executed obt akshay thanx team thoroughly enjo...,4,Anonymous,NaN,Very well executed obt by Akshay.... Thanx as ...
2,Aamby Valley City,19 Degree North,awesome experience atv tracts obstacles mainta...,5,Anonymous,NaN,Awesome experience at the ATV\nTracts and obst...
3,Aamby Valley City,19 Degree North,visited aamby valley yesterday short excursion...,4,Anonymous,NaN,we visited the Aamby Valley yesterday for shor...
4,Aamby Valley City,19 Degree North,far mumbai place finest adventure places visit...,5,Anonymous,NaN,"Not far from Mumbai, this place is one of the ..."


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482466 entries, 0 to 1482465
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   City        1482466 non-null  object 
 1   Place       1482466 non-null  object 
 2   Review      1482466 non-null  object 
 3   Rating      1482466 non-null  int64  
 4   Name        1482466 non-null  object 
 5   Date        0 non-null        float64
 6   Raw_Review  1482466 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 79.2+ MB


In [11]:
data.drop(["Date", "Name"], axis = 1, inplace = True)

In [12]:
data_pro_1 = data.groupby(["City", "Place"]).mean()
data_pro_1 = data_pro_1.reset_index()

<ipython-input-12-916c3e9871f3>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_pro_1 = data.groupby(["City", "Place"]).mean()


In [13]:
data_pro_1

,City,Place,Rating
0,Aamby Valley City,19 Degree North,4.588235
1,Abhaneri,Chand Baori (Step well),4.248331
2,Abhaneri,Harshat Mata Temple,3.915254
3,Acharapakkam,Achirupakkam,2.000000
4,Acharapakkam,Mazhai Malai Madha Shrine,5.000000
...,...,...,...
15234,Ziro,Shiva Linga,3.800000
15235,Ziro,Talley Valley Wildlife Sanctuary,3.333333
15236,Ziro,Tarin Fish Farm,4.000000
15237,Zuluk,Thambi View Point,4.142857


In [14]:
#Stop word removal

import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('wordnet')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [111]:
review = []

for i, j in zip(data_pro_1["City"].values[12362 : ], data_pro_1["Place"].values[12362 : ]):
  text = " ".join(data[(data["City"] == i) & (data["Place"] == j)].iloc[:, -1].values)
  #print(text)
  # Stop words removal
  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(text)
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  #Lemmatization of words
  lm = WordNetLemmatizer()
  lm_sentence = []
  for w in word_tokens:
    if w not in stop_words:
      lm_sentence.append(lm.lemmatize(w))

  # Frequency check of words and picking top 50 words for hashtags & wordcloud
  wd = WordCloud(max_font_size=100, max_words=50, background_color="white").generate(" ".join(lm_sentence))
  text_dictionary = wd.process_text(text)
  sst = {i : j for i, j in sorted(text_dictionary.items(),reverse=True, key=lambda item: item[1]) if len(i) > 1}
  key = list(sst.keys())[:50]
  review.append(" ".join(key))

In [114]:
data_pro_1["Short crisp"] = review
data_pro_1["Rating"] = data_pro_1["Rating"].apply(lambda x : round(x, 1))

In [119]:
data_pro_1

,City,Place,Rating,Short crisp
0,Aamby Valley City,19 Degree North,4.6,place ATV time Aamby Valley experience adventu...
1,Abhaneri,Chand Baori (Step well),4.2,Jaipur step well place one Agra visit well ste...
2,Abhaneri,Harshat Mata Temple,3.9,temple Chand Baori visit Abhaneri step well Ha...
3,Acharapakkam,Achirupakkam,2.0,Lord Ganesha starting ritual Hindus common bel...
4,Acharapakkam,Mazhai Malai Madha Shrine,5.0,shrine one hill situated Tamil Nadu history pl...
...,...,...,...,...
15234,Ziro,Shiva Linga,3.8,place Shiva Ziro linga temple Lord road local ...
15235,Ziro,Talley Valley Wildlife Sanctuary,3.3,Ziro forest Talley bamboo flora Valley sanctua...
15236,Ziro,Tarin Fish Farm,4.0,fish farm cultivation Ziro Tarin paddy place o...
15237,Zuluk,Thambi View Point,4.1,view Zuluk Thambi point see Silk Route place r...


In [137]:
from numpy import NaN
state_name = {}
city_lat = {}
city_lon = {}
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="google", timeout = 2)
for i in data_pro_1["City"].unique()[1393 : ]:
  try:
    country = "India"
    loc = geolocator.geocode(i + ", "+ country)
    st = loc.raw["display_name"].split(",")
    if (st[-2].strip()).isdigit():
      state_name[i] = st[-3]
      city_lat[i] = loc.raw["lat"]
      city_lon[i] = loc.raw["lon"]
    else:
      state_name[i] = st[-2]
      city_lat[i] = loc.raw["lat"]
      city_lon[i] = loc.raw["lon"]  
  except AttributeError:
    state_name[i] = NaN
    city_lat[i] = NaN
    city_lon[i] = NaN

In [140]:
data_pro_1["State"] = data_pro_1["City"].map(state_name)
data_pro_1["City Latitude"] = data_pro_1["City"].map(city_lat)
data_pro_1["City Longitude"] = data_pro_1["City"].map(city_lon)

In [146]:
data_pro_1.to_csv("Indian_Tourism_custamized.csv")

In [148]:
data_pro_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15239 entries, 0 to 15238
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   City            15239 non-null  object 
 1   Place           15239 non-null  object 
 2   Rating          15239 non-null  float64
 3   Short crisp     15239 non-null  object 
 4   State           15079 non-null  object 
 5   City Latitude   15079 non-null  object 
 6   City Longitude  15079 non-null  object 
dtypes: float64(1), object(6)
memory usage: 833.5+ KB


# **Final processed data is uploaded in my github.**